# This notebook is used to study:
#### - How the distance of the reconstructed shower start point improves when using PPN instead of the previous ('umbrella') method

In [1]:
import sys
sys.path.append("/home/rberner/cernbox/PhD/pi0_reconstruction/reco_software/lartpc_mlreco3d")   # locally
sys.path.append("/home/rberner/cernbox/PhD/pi0_reconstruction/reco_software/pi0_reco")          # locally
#sys.path.append("/u/nu/rberner/pi0_reconstruction/lartpc_mlreco3d")                            # nu-gpu0x
#sys.path.append("/u/nu/rberner/pi0_reconstruction/pi0_reco")                                   # nu-gpu0x
#sys.path.append("../lartpc_mlreco3d")                                                          # JupyterLab
#sys.path.append("../pi0_reco")                                                                 # JupyterLab

In [2]:
io_cfg = '''
iotool:
  batch_size: 1
  shuffle: False
  num_workers: 4
  collate_fn: CollateSparse
  dataset:
    name: LArCVDataset
    data_keys:
      #- /gpfs/slac/staas/fs1/g/neutrino/kterao/data/mpvmpr_2020_01_v04/test.root
      - ../data/test.root
    limit_num_files: 1
    schema:
      input_data:
        - parse_sparse3d_scn
        - sparse3d_pcluster
      segment_label:
        - parse_sparse3d_scn
        - sparse3d_pcluster_semantics
      semantics:
        - parse_sparse3d_scn
        - sparse3d_pcluster_semantics
      dbscan_label:
        - parse_cluster3d_clean_full
        - cluster3d_pcluster
        - particle_corrected
        - sparse3d_pcluster_semantics
      particles_label:
        - parse_particle_points
        - sparse3d_pcluster
        - particle_corrected
      particles:
        - parse_particle_asis
        - particle_corrected
        - cluster3d_pcluster
      cluster_label:
        - parse_cluster3d_full
        - cluster3d_pcluster
        - particle_corrected
'''

# Convert the string to a dictionary
#import yaml
#cfg = yaml.load(cfg,Loader=yaml.Loader)

# Pre-process configuration
#from mlreco.main_funcs import process_config
#process_config(cfg)

In [3]:
from mlreco.main_funcs import prepare, apply_event_filter
from mlreco.utils.gnn.evaluation import node_assignment, node_assignment_bipartite, edge_assignment_score, clustering_metrics, node_assignment_score

ModuleNotFoundError: No module named 'mlreco'

In [ ]:
# Configuration of the reconstruction chain
chain_cfg = '''
name: pi0_chain_fiducialized
#net_cfg: /home/rberner/cernbox/PhD/pi0_reconstruction/reco_software/config_files/fullChain_gnn_dbscan_cpu_X1.cfg      # locally
#net_cfg: /u/nu/rberner/pi0_reconstruction/config_files/fullChain_gnn_dbscan_cpu.cfg                                   # nu-gpu0x
#net_cfg: ../config_files/fullChain_gnn_dbscan_gpu0.cfg                                                                # JupyterLab
net_cfg: ../config_files/fullChain_gnn_dbscan_cpu.cfg                                                                  # JupyterLab
segment:                   uresnet                     # label, uresnet
deghost:                   label                       # label, uresnet
charge2energy:                                         # null, constant, average(, full, enet)
charge2energy_cst:         0.0082                      # energy response constant (0.0082 for mask, 0.0052 for uresnet)
charge2energy_average:     0.877                       # energy response average (1.3693 for mask, 0.877 for uresnet)
shower_start:              gnn                         # label, ppn, gnn (if using gnn, use uresnet)
shower_fragment:           gnn                         # label, dbscan, gnn
shower_direction:          pca                         # label, pca, cent
shower_cluster:            gnn                         # label, cone, gnn
shower_cluster_params:
  IP: 40
  Distance: 300
shower_energy:             pixel_sum                   # label, pixel_sum
shower_match:              ppn                         # label, proximity, ppn
refit_dir:                 true                        # true, false
refit_cone:                true                        # true, false
fiducialize:               14                          # Number of pixels to be removed from edge of LAr volume (pixel pitch: 3mm)
PPN_score_thresh:          [0.9, 0.9, 0.9, 0.9, 0.9]   # For PPN predictions to be classified as shower, track, michel, delta or LEScatter, respectively
'''
import yaml
chain_cfg = yaml.load(chain_cfg,Loader=yaml.Loader)

In [ ]:
# Progress bar
from IPython.display import HTML, display
def progress(count, total, unit, message=''):
    return HTML("""
        <progress 
            value='{count}'
            max='{total}',
            style='width: 30%'
        >
            {count}
        </progress> {count}/{total} {unit} ({frac}%) ... {message}
    """.format(count=count, total=total, unit=unit, frac=int(float(count)/float(total)*100.),message=message))

In [ ]:
class Results(object):
    '''
    Class for the storage of result acquired during chain.run_loop() process.
    The attributes are added in a dynamic way.
    This class can be used as:
    res = Results()
    res.attribute = someobject
    Use Results.__dict__ to show attributes.
    '''

    # Defined attributes
    # -------------------------------------
    '''
    ev_id

    # FOR ALL TRUE PI0 DECAYS IN THE EVEN
    true_n_pi0s
    true_n_gammas
    true_pi0_track_ids
    true_gamma_particle_ids
    true_gamma_group_ids
    true_gamma_mom
    true_gamma_dir
    true_gamma_first_step
    true_gamma_pos
    true_gamma_ekin
    true_gamma_edep
    true_gamma_voxels
    true_gamma_n_voxels
    true_OOFV                   # If at least one edep is at boundary pixels -> store shower_number in this list
    true_gamma_angle
    true_pi0_mass

    # FOR ALL SHOWER INSTANCES WHICH GOT MATCHED TO A PI0 DECAY IN THE EVEN
    reco_n_pi0
    reco_n_gammas
    reco_matches
    reco_gamma_mom
    reco_gamma_dir
    reco_gamma_start
    reco_gamma_edep
    reco_gamma_pid
    reco_gamma_voxels_mask      # array of n lists (n=number of reco showers) which contain voxel_mask indices
    reco_gamma_n_voxels_mask    # length of gamma_voxels_mask
    reco_gamma_voxels           # array of n lists (n=number of reco showers) which contain voxel coordinates
    reco_gamma_n_voxels         # length of gamma_voxels
    reco_OOFV                   # If at least one edep is at boundary pixels -> store shower_number in this list
    reco_gamma_angle
    reco_pi0_mass
    
    # FOR ALL SHOWER CLUSTER INSTANCES IN THE EVENT
    clusters_n_clusters
    clusters_start
    clusters_dir
    clusters_voxels
    clusters_energy
    '''
    
    pass

In [ ]:
# Some imports
from larcv import larcv
import numpy as np
from numpy import linalg
from pi0.directions.estimator import FragmentEstimator, DirectionEstimator # explicitly used for the direction estimation

In [ ]:
# Initialize the chain
from pi0.chain import Pi0Chain
chain = Pi0Chain(io_cfg, chain_cfg)

In [ ]:
# Define a list with the results obtained by run_loop()
ResultsList = []


# Loop over dataset
data_size = 40 #len(chain.hs.data_io)
out       = display(progress(0,data_size,'images'),display_id=True)

for event in range(data_size):
    
    chain.run_loop()


    # Instantiate Results class
    extracted_data = Results()


    # Add members of the Results class
    extracted_data.event_id                    = chain.true_info['ev_id']
    assert chain.true_info['ev_id']            == chain.event['index']

    extracted_data.true_n_pi0s                 = chain.true_info['n_pi0']
    extracted_data.true_n_gammas               = chain.true_info['n_gammas']
    extracted_data.true_pi0_track_ids          = chain.true_info['pi0_track_ids']
    extracted_data.true_gamma_particle_ids     = chain.true_info['gamma_particle_ids']
    extracted_data.true_gamma_group_ids        = chain.true_info['gamma_group_ids']
    extracted_data.true_gamma_mom              = chain.true_info['gamma_mom']
    extracted_data.true_gamma_dir              = chain.true_info['gamma_dir']
    extracted_data.true_gamma_first_step       = chain.true_info['gamma_first_step']
    extracted_data.true_gamma_pos              = chain.true_info['gamma_pos']
    extracted_data.true_gamma_ekin             = chain.true_info['gamma_ekin']
    extracted_data.true_gamma_edep             = chain.true_info['gamma_edep']
    extracted_data.true_gamma_voxels           = chain.true_info['gamma_voxels']
    extracted_data.true_gamma_n_voxels         = chain.true_info['gamma_n_voxels']
    extracted_data.true_OOFV                   = chain.true_info['OOFV']
    extracted_data.true_gamma_angle            = chain.true_info['gamma_angle'] # [rad]
    extracted_data.true_pi0_mass               = chain.true_info['pi0_mass']

    extracted_data.reco_n_pi0s                 = chain.reco_info['n_pi0']
    extracted_data.reco_n_gammas               = chain.reco_info['n_gammas']
    extracted_data.reco_matches                = chain.reco_info['matches']
    extracted_data.reco_gamma_mom              = chain.reco_info['gamma_mom']
    extracted_data.reco_gamma_dir              = chain.reco_info['gamma_dir']
    extracted_data.reco_gamma_start            = chain.reco_info['gamma_start']
    extracted_data.reco_gamma_edep             = chain.reco_info['gamma_edep']
    extracted_data.reco_gamma_pid              = chain.reco_info['gamma_pid']
    extracted_data.reco_gamma_voxels_mask      = chain.reco_info['gamma_voxels_mask']
    extracted_data.reco_gamma_n_voxels_mask    = chain.reco_info['gamma_n_voxels_mask']
    extracted_data.reco_gamma_voxels           = chain.reco_info['gamma_voxels']
    extracted_data.reco_gamma_n_voxels         = chain.reco_info['gamma_n_voxels']
    extracted_data.reco_OOFV                   = chain.reco_info['OOFV']
    extracted_data.reco_gamma_angle            = chain.reco_info['gamma_angle'] # [rad]
    extracted_data.reco_pi0_mass               = chain.reco_info['pi0_mass']
    
        
    #for sh in range(extracted_data.true_n_gammas):
        #print(' true start: ', extracted_data.true_gamma_first_step[sh])
        #print(' true dir: ', extracted_data.true_gamma_dir[sh])
        #print(' ------------------------------------------------ ')


    extracted_data.clusters_n_clusters = len(chain.output['showers'])
    extracted_data.clusters_start      = []
    extracted_data.clusters_dir        = []
    extracted_data.clusters_voxels     = []
    extracted_data.clusters_energy     = []
    for i, shower in enumerate(chain.output['showers']):
        extracted_data.clusters_start.append(shower.start)
        extracted_data.clusters_dir.append(shower.direction)
        extracted_data.clusters_voxels.append(shower.voxels)
        extracted_data.clusters_energy.append(shower.energy)


    # Append the extracted data to the ResultsList
    ResultsList.append(extracted_data)
    
    
    
    # Select events with exactly 1 true pi0 -> gamma+gamma and two 
    #if chain.true_info['n_gammas'] != 2:
    #    out.update(progress(event,data_size,'images'))
    #    continue
    #if len(chain.output['showers']) != 2:
    #    out.update(progress(event,data_size,'images'))
    #    continue

    
 

    # Draw event
    '''
    # Draw the last event if requested
    #i_draw    = 0
    #n_draw    = 0
    #if 'matches' in chain.output and len(chain.output['matches']) and i_draw < n_draw:
    #    chain.draw()
    #    i_draw += 1
    #chain.draw()
    draw_eventIDs = []
    if chain.true_info['ev_id'] in draw_eventIDs:
        chain.draw()
    '''
    #try:
    #    chain.draw()
    #except:
    #    pass

    #draw_eventIDs = [1,2,53,54,68,70,71,73,76,78,79,80,83,84] # 1,2,3,4,5,6,7,8,9,10
    #if chain.true_info['ev_id'] in draw_eventIDs:
    #    chain.draw()
    
    #chain.draw()
    
    
    # Print to screen
    print_information = False
    if print_information:
        #if extracted_data.true_n_pi0s>0 or extracted_data.reco_n_pi0s>0:
        print(' event_id: \t \t ',           extracted_data.event_id)
        #print(' true_n_pi0s: \t \t ',        extracted_data.true_n_pi0s)
        #print(' reco_n_pi0s: \t \t ',        extracted_data.reco_n_pi0s)
        #print(' clusters_n_clusters: \t ',   extracted_data.clusters_n_clusters)
        #print(' true_gamma_edep: \t ',       extracted_data.true_gamma_edep)
        #print(' reco_gamma_edep: \t ',       extracted_data.reco_gamma_edep)
        #print(' true_gamma_dir: \t ',        extracted_data.true_gamma_dir)
        #print(' reco_gamma_dir: \t ',        extracted_data.reco_gamma_dir)
        #print(' clusters_dir: \t \t ',       extracted_data.clusters_dir)
        #print(' true_gamma_pos: \t ',        extracted_data.true_gamma_pos)
        #print(' true_gamma_first_step:\t ',  extracted_data.true_gamma_first_step)
        #print(' reco_gamma_start: \t ',      extracted_data.reco_gamma_start)
        #print(' clusters_start: \t ',        extracted_data.clusters_start)
        #print(' cluster_start: ')
        #for index in range(extracted_data.clusters_n_clusters):
        #    print(' \t \t \t ', extracted_data.clusters_start[index])
        #print(' true_gamma_angle [rad]: ',   extracted_data.true_gamma_angle)
        #print(' reco_gamma_angle [rad]: ',   extracted_data.reco_gamma_angle)
        #print(' true_OOFV: \t \t ',          extracted_data.true_OOFV)
        #print(' reco_OOFV: \t \t ',          extracted_data.reco_OOFV)
        #print(' reco_pi0_mass: \t ',         extracted_data.reco_pi0_mass)
        #print(' ------------------------------------------------------ ')
    
    ##for clust in range(extracted_data.clusters_n_clusters):
    ##    print(' clust: ', clust)
    ##    print(len(chain.output['forward']['points'][0][clust][:]))
    
    
    
    #print(' ------------------------------------------------------------------------------------------- ')

    # Update progress bar
    out.update(progress(event,data_size,'images'))
out.update(progress(data_size,data_size,'images'))

# Assign clusters <---> true showers

In [ ]:
class AnalysedData(object):
    '''
    Class for the storage of variables of events which passed the selection.
    The attributes are added in a dynamic way.
    This class can be used as:
    ana = AnalysedData()
    ana.attribute = someobject
    Use AnalysedData.__dict__ to show attributes.
    '''

    # Defined attributes
    # -------------------------------------
    '''
    event_id
    angle            # angle between true_gamma and reconstructed cluster_gamma [deg]
    true_1step
    reco_start
    true_ekin
    true_edep
    reco_edep        # energy of the reconstructed cluster_gamma [MeV]
    '''
    pass

In [4]:
AnalysisList = []

for i, res in enumerate(ResultsList):
    
    # Create matrix for distances between cluster_i and true_shower_j:
    #if (res.clusters_n_clusters==0 or res.true_n_gammas==0):
        #print(' \t skip, n_clust: ', res.clusters_n_clusters, ' \t true_n_gammas: ', res.true_n_gammas)
        #continue
    
    
    
    
    # Event Selection
    if (res.clusters_n_clusters != 2): continue
    if (res.true_n_gammas != 2): continue

        
        
        
        
        
        
    # CUT: WANT TO HAVE SAME NUMBER OF TRUE GAMMAS AND TRUE CLUSTERS
    #if res.true_n_gammas != 2 or res.clusters_n_clusters != 2:
    #    print(' \t skip, true_n_gammas: ', res.true_n_gammas, ' \t n_clust: ', res.clusters_n_clusters)
    #    continue

    #print(' event_id: ', res.event_id)
    #print(' clusters_n_clusters: ', res.clusters_n_clusters)
    #print(' true_n_gammas: ', res.true_n_gammas)
    
    dist_mat = np.zeros([res.clusters_n_clusters, res.true_n_gammas]) # columns = number of clusters, rows = number of true_gammas

    #print(' \nclusters start positions: ')
    #for cl in range(res.clusters_n_clusters):
        #print(res.clusters_start[cl])

    #print(' \ntrue gammas start positions: ')
    #for gamma in range(res.true_n_gammas):
        #print(res.true_gamma_first_step[gamma])
    
    for cluster in range(res.clusters_n_clusters):
        for true_gamma in range(res.true_n_gammas):
            dist_mat[cluster][true_gamma] = np.linalg.norm(res.clusters_start[cluster]-res.true_gamma_first_step[true_gamma])
            
    #print(' \ndist_mat: ')
    #print(dist_mat)
    #print(' \n ')

    # Store assignments
    assignments = [] # list of U arrays (where U is min(n_clusters,n_true_gammas)). First array entry: cluster index, second array: true_gamma index.
    for i in range(min(dist_mat.shape)):
        indices = np.argwhere(dist_mat.min() == dist_mat)

        assignments.append(indices[0])

        dist_mat[indices[0][0],:] = 999
        dist_mat[:,indices[0][1]] = 999

    # Calculate dot products between assigned showers
    for i, ass in enumerate(assignments):
        #print(' i: ', i)
        analysis_data = AnalysedData()
        analysis_data.event_id   = res.event_id
        analysis_data.angle      = np.arccos(np.dot(res.clusters_dir[ass[0]],res.true_gamma_dir[ass[1]]))*360/(2*np.pi)
        analysis_data.true_1step = res.true_gamma_first_step[ass[1]]
        analysis_data.reco_start = res.clusters_start[ass[0]]
        analysis_data.true_ekin  = res.true_gamma_ekin[ass[1]]
        analysis_data.true_edep  = res.true_gamma_edep[ass[1]]
        analysis_data.reco_edep  = res.clusters_energy[ass[0]]
        #print(' event_id: \t \t \t ',            analysis_data.event_id)
        #print(' angle [deg]: \t \t \t ',         analysis_data.angle)
        #print(' true gamma 1step: \t \t ',       analysis_data.true_1step)
        #print(' reco gamma start: \t \t ',       analysis_data.reco_start)
        #print(' true gamma energy [MeV]: \t ',    analysis_data.true_ekin)
        #print(' true edep [MeV]: \t \t ',         analysis_data.true_edep)
        #print(' cluster energy [MeV]: \t \t ',    analysis_data.reco_edep)
        
        AnalysisList.append(analysis_data)
    #print(' ------------------------------------------------------------------------------- ')    



NameError: name 'ResultsList' is not defined

In [ ]:
import math

angles     = []
true_1step = []
reco_start = []
distance   = []
true_ekin  = []
true_edep  = []
reco_edep  = []

PIXEL_PITCH = 3. # [mm]

for i, entry in enumerate(AnalysisList):
   
    #if math.isnan(entry.angle) or math.isnan(entry.true_ekin) or math.isnan(entry.true_edep) or math.isnan(entry.reco_edep):
    #    print(' NAN in any of the variables -> skip event', i)
    #    continue
        

    angles.append(entry.angle)
    true_1step.append(entry.true_1step)
    reco_start.append(entry.reco_start)
    _dist = np.linalg.norm(entry.reco_start - entry.true_1step) / PIXEL_PITCH
    distance.append(_dist) #(entry.reco_start[0]-) + (entry.reco_start-) + (entry.reco_start-)
    true_ekin.append(entry.true_ekin)
    true_edep.append(entry.true_edep)
    reco_edep.append(entry.reco_edep)
    
    #print(' i: ', i)
    #print(' entry:      ', entry)
    #print(' event_id:   ', entry.event_id)
    #print(' angle:      ', entry.angle) 
    #print(' true_1step: ', entry.true_1step)
    #print(' reco_start: ', entry.reco_start)
    #print(' distance:   ', _dist)
    #####print(entry.event_id, ',', _dist)
    #print(' true ekin:  ', entry.true_ekin)
    #print(' true edep:  ', entry.true_edep)
    #print(' reco edep:  ', entry.reco_edep)

# Plot distances

In [ ]:
import pandas as pd
import seaborn
from matplotlib import pyplot as plt

seaborn.set(rc={'figure.figsize':(15, 10),})
seaborn.set_context('talk') # or paper


# Define histogram range and binning
x_min    = 0
x_max    = 10
n_bins_x = 100
x_bins = np.linspace(x_min,x_max,n_bins_x+1)


# Define parameters of the frame
fig = plt.figure() # plt.figure(figsize=(width,height))
#fig.patch.set_facecolor('white')
#fig.patch.set_alpha(0.0)
ax = fig.add_subplot(111)
#ax.patch.set_facecolor('#ababab') # #ababab
ax.patch.set_alpha(0.0)
ax.spines['bottom'].set_color('0.5') #'black', ...
ax.spines['bottom'].set_linewidth(2)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_color('0.5')
ax.spines['top'].set_linewidth(2)
ax.spines['top'].set_visible(True)
ax.spines['right'].set_color('0.5')
ax.spines['right'].set_linewidth(2)
ax.spines['right'].set_visible(True)
ax.spines['left'].set_color('0.5')
ax.spines['left'].set_linewidth(2)
ax.spines['left'].set_visible(True)


# Ticks, grid and ticks labels
ax.tick_params(direction='in', length=10, width=2,                  # direction, length and width of the ticks (in, out, inout)
                colors='0.5',                                       # color of the ticks ('black', '0.5')
                bottom=True, top=True, right=True, left=True,       # whether to draw the respective ticks
                zorder = 10.,                                       # tick and label zorder
                pad = 10.,                                          # distance between ticks and tick labels
                labelsize = 17,                                     # size of the tick labels
                labelright=False, labeltop=False)                   # wether to draw the tick labels on axes
                #labelrotation=45.                                  # rotation of the labels
                #grid_color='black',                                # grid
                #grid_alpha=0.0,
                #grid_linewidth=1.0,
# colors='black','0.5'


# Plot distances
n, bins, patches = plt.hist(distance, bins=n_bins_x, range=[x_min,x_max], histtype='stepfilled', stacked=True, linewidth=3, alpha=0.8)

# Legend
#unfiducialized = 'not fiducialized'
#entry_0 = 'Entry 0'
#entry_1 = 'Entry 1'
#plt.legend([entry_1, entry_0], loc=[0.4,0.65], prop={'size': 17}) # loc='upper right'


# Axis labels
plt.xlabel('Distance true $\gamma$ 1st step to reco cluster start [pixel pitches]', fontsize=20, labelpad=20)
plt.ylabel('Entries [-]', fontsize=20, labelpad=20)


#plt.yscale('log') # linear, log
plt.yscale('symlog', linthreshy=1)


#plt.axvline(134.9770, color='r', linestyle='dashed', linewidth=2)


# Save figure
#fig_name = '0' + folder + '_pi0_mass_peak.png'
fig_name = 'distance_true_1st_step_to_reco_start_10000ev_method_old.png'
#plt.savefig(fig_name, dpi=400) # bbox_inches='tight'
#plt.show()

# Reading .csv file and make plot

In [ ]:
import numpy as np
import pandas as pd
import seaborn
from matplotlib import pyplot as plt

seaborn.set(rc={'figure.figsize':(15, 10),})
seaborn.set_context('talk') # or paper


# Define histogram range and binning
x_min    = -1
x_max    = 1
n_bins_x = 40
x_bins = np.linspace(x_min,x_max,n_bins_x+1)


# Define parameters of the frame
fig = plt.figure() # plt.figure(figsize=(width,height))
#fig.patch.set_facecolor('white')
#fig.patch.set_alpha(0.0)
ax = fig.add_subplot(111)
#ax.patch.set_facecolor('#ababab') # #ababab
ax.patch.set_alpha(0.0)
ax.spines['bottom'].set_color('0.5') #'black', ...
ax.spines['bottom'].set_linewidth(2)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_color('0.5')
ax.spines['top'].set_linewidth(2)
ax.spines['top'].set_visible(True)
ax.spines['right'].set_color('0.5')
ax.spines['right'].set_linewidth(2)
ax.spines['right'].set_visible(True)
ax.spines['left'].set_color('0.5')
ax.spines['left'].set_linewidth(2)
ax.spines['left'].set_visible(True)


# Ticks, grid and ticks labels
ax.tick_params(direction='in', length=10, width=2,                  # direction, length and width of the ticks (in, out, inout)
                colors='0.5',                                       # color of the ticks ('black', '0.5')
                bottom=True, top=True, right=True, left=True,       # whether to draw the respective ticks
                zorder = 10.,                                       # tick and label zorder
                pad = 10.,                                          # distance between ticks and tick labels
                labelsize = 17,                                     # size of the tick labels
                labelright=False, labeltop=False)                   # wether to draw the tick labels on axes
                #labelrotation=45.                                  # rotation of the labels
                #grid_color='black',                                # grid
                #grid_alpha=0.0,
                #grid_linewidth=1.0,
# colors='black','0.5'


# Define dataframes
#df = pd.read_csv(chain_cfg['name']+'_log.csv')
#print(df1.to_string())
df = pd.read_csv('distance_true_1st_step_to_reco_start.csv')

# Plot dataframes
#n, bins, patches = plt.hist(df.dist_true_1st_step_to_cluster_start__method_PPN_pixel_pitches, bins=n_bins_x, range=[x_min,x_max], histtype='step', color='r', linewidth=3, alpha=0.7)
n, bins, patches = plt.hist(df.dist_method_old_minus_dist_method_PPN_pixel_pitches, bins=n_bins_x, range=[x_min,x_max], histtype='step', color='r', linewidth=3, alpha=0.7)

# Axis labels
plt.xlabel('Distance Previous-Method - PPN-Method [pixel pitches]', fontsize=20, labelpad=20)
plt.ylabel('Entries [-]', fontsize=20, labelpad=20)


#plt.yscale('log') # linear, log
#plt.yscale('symlog', linthreshy=1)


#plt.axvline(134.9770, color='r', linestyle='dashed', linewidth=2)


# Save figure
#fig_name = '0' + folder + '_pi0_mass_peak.png'
fig_name = 'distance_true_1st_step_to_reco_start_10000ev_method_old_minus_method_PPN.png'
#plt.savefig(fig_name, dpi=400) # bbox_inches='tight'
#plt.show()